<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch03/03_karaage_ai_book_generate_text_markov_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルコフ連鎖を活用した文章生成

## 教師データのダウンロード

In [1]:
!wget https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
!unzip 2093_ruby_28087.zip

--2020-10-20 17:38:10--  https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip [following]
--2020-10-20 17:38:11--  https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421747 (412K) [application/zip]
Saving to: ‘2093_ruby_28087.zip’

2093_ruby_28087.zip 100%[===================>] 411.86K  --.-KB/s    in 0.07s   

2020-10-20 17:38:11 (5.45 MB/s) - ‘

ファイルを読み込みます

In [2]:
text_list = []
with open('dogura_magura.txt', encoding='shift_jis') as f:
  text_list = f.readlines()

In [3]:
text_list[0:10]

['ドグラ・マグラ\n',
 '夢野久作\n',
 '\n',
 '-------------------------------------------------------\n',
 '【テキスト中に現れる記号について】\n',
 '\n',
 '《》：ルビ\n',
 '（例）蜜蜂《みつばち》\n',
 '\n',
 '｜：ルビの付く文字列の始まりを特定する記号\n']

## データの前処理


不要な文字の削除

In [5]:
import re
def normalize_text(text):
  text = re.sub(r'》', '', text)
  text = re.sub(r'※', '', text)
  text = re.sub(r'《', '', text)
  text = re.sub(r'［', '', text)
  text = re.sub(r'＃', '', text)
  text = re.sub(r'-', '', text)
  text = re.sub(r'｜', '', text)
  text = re.sub(r'］', '', text)
  text = re.sub(r'［', '', text)
  text = re.sub(r'【','', text)
  text = re.sub(r'】','', text)
  text = text.strip()
  return text

In [6]:
new_text_list = []
for text in text_list:
  text = normalize_text(text)
  new_text_list.append(text)

text_list = new_text_list

In [7]:
text_list[0:10]

['ドグラ・マグラ',
 '夢野久作',
 '',
 '',
 'テキスト中に現れる記号について',
 '',
 '：ルビ',
 '（例）蜜蜂みつばち',
 '',
 '：ルビの付く文字列の始まりを特定する記号']

形態素解析ライブラリの「janome」をインストール

In [4]:
!pip install janome

     |████████████████████████████████| 19.7MB 1.3MB/s 


分かち書き

In [8]:
from janome.tokenizer import Tokenizer

In [9]:
def wakachigaki(text_list):
  t = Tokenizer()
  words = []
  for text in text_list:
    tokens = t.tokenize(text)
    for token in tokens:
      pos = token.part_of_speech.split(',')[0]
      words.append(token.surface)

  text = ' '.join(words)
  return text

In [10]:
word_list = [w for w in wakachigaki(text_list).split()]

In [11]:
word_list[0:10]

['ドグラ・マグラ', '夢野', '久作', 'テキスト', '中', 'に', '現れる', '記号', 'について', '：']

## マルコフ連鎖モデルの学習

### 簡単な例で確認

In [12]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。']

一階のマルコフ連鎖モデル

In [13]:
def make_markov_model_1(word_list):
  markov = {}
  w1 = ''
  for word in word_list:
    if w1:
      if w1 not in markov:
        markov[w1] = []
      markov[w1].append(word)
    w1 = word
  return markov  

In [14]:
markov_model_test_1 = make_markov_model_1(test_word_list)

学習したモデルの確認

In [16]:
def check_model(model, check_numb=10):
  count = 0
  for key in model.keys():
    if count >= 0:
      print('key:', key)
      print('value:', model[key])
      print('------------------------')
    count += 1
    if count > check_numb:
      break

In [17]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------


In [18]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。空が青い。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。',
 '空',
 'が',
 '青い',
 '。']

In [19]:
markov_model_test_1 = make_markov_model_1(test_word_list)

In [20]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き', '青い']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私', '空']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------
key: 空
value: ['が']
------------------------
key: 青い
value: ['。']
------------------------


In [21]:
def make_markov_model_2(text_list):
  markov = {}
  w1 = ''
  w2 = ''
  for word in text_list:
    if w1 and w2:
      if (w1, w2) not in markov:
        markov[(w1, w2)] = []
      markov[(w1, w2)].append(word)
    w1, w2 = w2, word
  return markov  

In [22]:
markov_model_test_2 = make_markov_model_2(test_word_list)

In [23]:
check_model(markov_model_test_2, check_numb=20)

key: ('私', 'は')
value: ['からあげ', 'おやつ']
------------------------
key: ('は', 'からあげ')
value: ['が', 'を']
------------------------
key: ('からあげ', 'が')
value: ['好き']
------------------------
key: ('が', '好き')
value: ['だ', 'だ']
------------------------
key: ('好き', 'だ')
value: ['。', '。']
------------------------
key: ('だ', '。')
value: ['君', '空']
------------------------
key: ('。', '君')
value: ['は']
------------------------
key: ('君', 'は')
value: ['からあげ']
------------------------
key: ('からあげ', 'を')
value: ['食べる']
------------------------
key: ('を', '食べる')
value: ['。']
------------------------
key: ('食べる', '。')
value: ['私']
------------------------
key: ('。', '私')
value: ['は']
------------------------
key: ('は', 'おやつ')
value: ['が']
------------------------
key: ('おやつ', 'が')
value: ['好き']
------------------------
key: ('。', '空')
value: ['が']
------------------------
key: ('空', 'が')
value: ['青い']
------------------------
key: ('が', '青い')
value: ['。']
------------------------


### ドグラ・マグラを使ってモデルを生成

In [24]:
markov_model_2 = make_markov_model_2(word_list)

In [25]:
check_model(markov_model_2, check_numb=20)

key: ('ドグラ・マグラ', '夢野')
value: ['久作']
------------------------
key: ('夢野', '久作')
value: ['テキスト', '全集']
------------------------
key: ('久作', 'テキスト')
value: ['中']
------------------------
key: ('テキスト', '中')
value: ['に']
------------------------
key: ('中', 'に')
value: ['現れる', 'ハッキリ', '、', '、', '走り込み', 'ヨボヨボ', '含ま', 'は', '納まっ', '凝固', 'ゴチャゴチャ', '落ち', '、', '追い込ん', '映っ', '突立', '、', '誘い込ん', '湧き', '落し', '並ん', 'は', '封じ込め', '、', '一つ', '、', 'は', '盛込ま', '、', '記述', '記述', '陳列', '含ま', '、', '潜伏', '引寄', 'タッタ', '含ま', 'は', 'は', '摘発', '、', '…', 'は', '落し', '胚胎', '尊', '。', 'は', 'も', '公', '並ぶ', 'は', '交', 'さまよう', '重なる', '。', '居る', 'トグロ', '立て', '立ち止まっ', '発見', '宿っ', '在る', '呼吸', '、', 'ホッ', '、', '、', '葬り', '立往生', '詰めかけ', 'は', '引っ', 'も', '平等', '往々', '、', '見知っ', 'は', '、', '潜在', '寝', '潜在', '包み込ん', '現われ', '含ま', '咲く', '辛', '刺', '落付', '蝨', 'サッと', '留め', '含ま', 'は', '並べ立て', '据え', 'は', '突込', 'は', 'は', '、', 'は', '居る', '居る', '立ち', 'ベタ', 'も', '、', '現われ', '、', 'は', '隠れ', 'は', '息', '盛ら', '、', '潜ん', '納まっ', '一', 'も', '潜ん', 'も', 'は'

## 文章の生成

### 2階のマルコフモデルで文章生成
作成したモデルを使って文章を生成する

In [26]:
import random

def generate_text_2(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2 = w2, tmp
    except:
      w1, w2  = random.choice(list(model.keys()))

  return sentence

In [27]:
print(generate_text_2(markov_model_2, 10))

大変だ。
彼奴きゃつの使う手は桑名くわなの何とかいう学者があっても宜しい位であるが、こんなに違って来るからね……弱い者苛いじめ付けられたり、首と称せらるる怪談なり。
而しかし、血に染まった鍬を下したる事実に二重人格的夢遊して部屋の隅に、吾輩の学説……その異様なので、青いメリンスの風呂敷を一刹那せつな……」「ヘヘヘイ。
先生は、今日までの気疲れが一時に照せば何であろう事業……そんなような緋縮緬ひぢりめんの湯沸器シンメルブッシュで御座いましょう。
又とほかに在りと雖いえども、その祭文歌の中に這入った。
思いますれどここらはまだ御不審なさるであろう、大卓子テーブルの端を押え付けている事実で、なおも悠々と落ち散って行って、その血統に絡み付いて、玄怪ニシテ捕捉シ難キノミ。
何か云うておられるようなもの）――――ンンンン……占めた。
前に述べた狐憑きなどの場合をいうとコンナ風に、両側に二つ並んでいるのでありながら、アノ通り夜と昼の日附を御覧になったので、私の横ッ面つらを突込んで眠り………事が……その女がい病院の診断は、普通の夢を見たる時、私はそれから徐おもむろに正木博士の心理の奥底の深い魅力をもってこの暗い、重苦しい水の如く渦巻き起って来る。
スワ又一大事と身も世もあらばこそだ。
無学文盲の農夫が発病後の夢中遊行中には学校の図書館、青空文庫作成ファイル：このファイル中で活躍して来た。



### 2階から5階のマルコフ連鎖に変更

In [28]:
def make_markov_model_5(word_list):
  markov = {}
  w1 = ''
  w2 = ''
  w3 = ''
  w4 = ''
  w5 = ''
  for word in word_list:
    if w1 and w2 and w3 and w4 and w5:
      if (w1, w2, w3, w4, w5) not in markov:
        markov[(w1, w2, w3, w4, w5)] = []
      markov[(w1, w2, w3, w4, w5)].append(word)
    w1, w2, w3, w4, w5 = w2, w3, w4, w5, word
  return markov

In [29]:
markov_model_5 = make_markov_model_5(word_list)

In [30]:
check_model(markov_model_5, 20)

key: ('ドグラ・マグラ', '夢野', '久作', 'テキスト', '中')
value: ['に']
------------------------
key: ('夢野', '久作', 'テキスト', '中', 'に')
value: ['現れる']
------------------------
key: ('久作', 'テキスト', '中', 'に', '現れる')
value: ['記号']
------------------------
key: ('テキスト', '中', 'に', '現れる', '記号')
value: ['について']
------------------------
key: ('中', 'に', '現れる', '記号', 'について')
value: ['：']
------------------------
key: ('に', '現れる', '記号', 'について', '：')
value: ['ルビ']
------------------------
key: ('現れる', '記号', 'について', '：', 'ルビ')
value: ['（']
------------------------
key: ('記号', 'について', '：', 'ルビ', '（')
value: ['例']
------------------------
key: ('について', '：', 'ルビ', '（', '例')
value: ['）']
------------------------
key: ('：', 'ルビ', '（', '例', '）')
value: ['蜜蜂']
------------------------
key: ('ルビ', '（', '例', '）', '蜜蜂')
value: ['みつば']
------------------------
key: ('（', '例', '）', '蜜蜂', 'みつば')
value: ['ち']
------------------------
key: ('例', '）', '蜜蜂', 'みつば', 'ち')
value: ['：']
------------------------
key: ('）', '蜜蜂', 'みつば', 'ち',

In [31]:
import random

def generate_text_5(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2, w3, w4, w5  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2, w3, w4, w5)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2, w3, w4, w5 = w2, w3, w4, w5, tmp
    except:
      w1, w2, w3, w4, w5  = random.choice(list(model.keys()))

  return sentence

In [32]:
print(generate_text_5(markov_model_5, 10))

なって行った。
「……お兄さまお兄さま……お兄さま……おにいさま……どうぞ……ね……」それは矢張やはり身体に釣り合わない、女みたような声で申しますと、唇をギリギリと噛んで、両手を握り固めてブルブルと慄ふるわして、眼を逆様さかさまに飛降りて自殺した患者の亀裂した頭蓋骨――――女房に擬して愛撫した枕と毛布製の人形――――手品を使うと称して、嚥下のみくだした真鍮煙管しんちゅうきせる――――素手すでで引裂いた錻力板ブリキいた――――女患者が捻じ曲げた檻房の鉄柵――ここで字下げ終わり「……アブナイッ……」「馬鹿ッ……」「……ハア……焚き殺すのがその頃の治療法だったのですね」「ところがこの淫仙先生はチットモ驚かない。
今度は方針を変えて婦女子の新葬を求め、夜陰に乗じて墓を発あばき、屍体を引きずり出して山の中に持って行こうとした。
……が……若林博士は、そうした私の態度を見守っていた。
がやがて傍近く歩み寄って来て、鍬を振り上げた右の肩をポンとたたいた。
呉一郎はそのまま、矢張りマジマジとした眼付きで正木博士の顔を、夢のように淡い言葉が、切れ切れに洩れ出した。
「……ところで、貴方あなたをこの部屋にお伴いたしました目的と申しますのは、要するにこの少年の鼻の穴は、曲りが少のう御座いますので、器械で覗きますと一直線に奥までわかる……お笑いになっては困ります。
落語では御座いませんか。
これは恐らくこの博士が、自身に何回となく変相をした経験があるせいでは御座いますまいか。
しかし若林博士の手腕が、如何に卓抜恐るべきものがあるかという事実の大要が理解されるであろうことを、吾輩は今一度、少女の寝顔を振り返った。



# 参考リンク
- https://omedstu.jimdofree.com/2018/05/06/%E3%83%9E%E3%83%AB%E3%82%B3%E3%83%95%E9%80%A3%E9%8E%96%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E7%94%9F%E6%88%90/
- https://qiita.com/k-jimon/items/f02fae75e853a9c02127